In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
import plotly.graph_objects as go
from typing import Tuple

In [5]:


def carregar_dados(filename: str) -> pd.DataFrame:
    dados = pd.read_csv(filename, sep=";", encoding="latin-1")
    dados['Data do pedido'] = pd.to_datetime(dados['Data do pedido'], errors='coerce')
    dados['Data de envio'] = pd.to_datetime(dados['Data de envio'], errors='coerce')
    dados['Quantidade'] = dados['Quantidade'].astype(int)
    dados['Vendas'] = pd.to_numeric(dados['Vendas'].str.replace(',', '.'))
    return dados

def tratamento_outliers(data: pd.Series, threshold: float = 3.0) -> pd.Series:
    mean = data.mean()
    std = data.std()
    is_outlier = (data - mean).abs() > threshold * std
    data_copy = data.copy()
    data_copy[is_outlier] = data.median()
    return data_copy

def dividir_dados(dados: pd.DataFrame) -> Tuple[np.ndarray, np.ndarray]:
    dados['Vendas'] = tratamento_outliers(dados['Vendas'])
    X = dados.index.astype('int64').values.reshape(-1, 1)
    y = dados['Vendas'].values
    return X, y

def treinar_modelo(X: np.ndarray, y: np.ndarray) -> XGBRegressor:
    tscv = TimeSeriesSplit(n_splits=5)
    
    param_grid = {
        'n_estimators': [100, 500, 1000],
        'learning_rate': [0.01, 0.1, 0.2],
        'max_depth': [5, 10, 15]
    }
    
    best_model = None
    best_mse = float('inf')
    
    for train_index, test_index in tscv.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        grid_search = GridSearchCV(XGBRegressor(objective='reg:squarederror', random_state=42), param_grid,
                                   cv=3, scoring='neg_mean_squared_error')
        grid_search.fit(X_train, y_train)
        model = grid_search.best_estimator_
    
        previsoes = model.predict(X_test)
    
        mse = mean_squared_error(y_test, previsoes) 
        print(f'MSE nos conjuntos de teste: {mse:.2f}')
    
        if mse < best_mse:
            best_mse = mse
            best_model = model
    
    return best_model

def criar_grafico(dados: pd.DataFrame, modelo: XGBRegressor) -> None:
    X, y = dividir_dados(dados)
    modelo.fit(X, y)
    previsoes = modelo.predict(X)
    df_resultados = pd.DataFrame({'Data': dados.index, 'Vendas_Real': y, 'Vendas_Previstas': previsoes})
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df_resultados['Data'], y=df_resultados['Vendas_Real'],
                             mode='lines', name='Vendas Reais', line=dict(color='blue')))
    
    fig.add_trace(go.Scatter(x=df_resultados['Data'], y=df_resultados['Vendas_Previstas'],
                             mode='lines', name='Previsões', line=dict(color='red')))
    
    fig.update_layout(
        title='Vendas - Previsao | Valor Real',
        xaxis_title='Data do Pedido',
        yaxis_title='Vendas',
        plot_bgcolor='black',
        paper_bgcolor='black',
        font=dict(color='white', size=18),
        showlegend=True,
        hovermode='x unified'
    )
    
    fig.update_layout(legend=dict(font=dict(size=16)))
    fig.update_layout(autosize=True)
    fig.show()
    
    for index, row in df_resultados.iterrows():
        data = row['Data']
        real = float(row['Vendas_Real'])
        previsto = float(row['Vendas_Previstas'])
        print(f'Data: {data}, Valor Real: {real:.2f}, Valor Previsto: {previsto:.2f}')



In [ ]:


filename = r'D:\seriestesteapp\super_loja.csv'
dados = carregar_dados(filename)
X, y = dividir_dados(dados)
modelo = treinar_modelo(X, y) 
criar_grafico(dados, modelo)
